In [79]:
import pandas as pd
import numpy as np
import re
import requests
import plotly
import html
import time
from datetime import datetime
import html2text

from bs4 import BeautifulSoup

ModuleNotFoundError: No module named 'html2text'

In [58]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs

# I used Firefox; you can use whichever browser you like.
browser = webdriver.Chrome()

# Tell Selenium to get the URL you're interested in.
browser.get("https://twitter.com/thejakartaglobe")

# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
        lastCount = lenOfPage
        time.sleep(1)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source

# Throw your source into BeautifulSoup and start parsing!
bs_data = bs(source_data)

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")




In [31]:
#bs_data

In [32]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('href="/')[1].split('/status')[0]

'CurtisSChin'

---

## Number of Tweets Captured

In [33]:
num_tweets = len(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'})))

In [34]:
num_tweets

0

---

## Handle

In [35]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('href="/')[1].split('/status')[0]

'CurtisSChin'

---

## Time

In [36]:
time_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split(' - ')[0]

---

## Date

Day

In [37]:
day_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[0]

Month

In [38]:
month_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[1]

Year

In [39]:
year_str = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[2]

---

## Convert to datetime object

In [40]:
from datetime import datetime

#datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [41]:
time_str.replace(" ", "")

'8:55AM'

In [42]:
datetime_object = datetime.strptime(month_str + ' ' + day_str + ' ' + year_str + ' ' + time_str.replace(" ", ""), '%b %d %Y %I:%M%p')

---

## Tweet

In [44]:
tweet = str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[0]).split('lang="en">')[1].split('<a class')[0].rstrip()

IndexError: list index out of range

---

## Check for keyword (tsunami)

In [ ]:
tweet = 'This is a tweet about a tsunami'

In [ ]:
'tsunami' in tweet

---

## Build Dictionary

In [119]:
temp_dict = {}

temp_dict['handle']     = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[0]).split('href="/')[1].split('/status')[0]
temp_dict['tweet']      = str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[0]).split('lang="en">')[1].split('<a class')[0].rstrip()
temp_dict['datetime']   = datetime_object
temp_dict['if_tsunami'] = 'tsunami' in temp_dict['tweet']

In [125]:
temp_dict

{'handle': 'thejakartaglobe',
 'tweet': 'China Pledges Easier Foreign Tourist Access to Tibet Amid US Pressure',
 'datetime': datetime.datetime(2019, 1, 13, 6, 25),
 'if_tsunami': False}

In [59]:
master_list = []
#loop_range = num_tweets - 1
loop_range = 10

for i in range(loop_range):
    temp_dict = {}

    temp_dict['handle']     = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('href="/')[1].split('/status')[0]
    
    try:
        temp_dict['tweet']      = str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[i]).split('lang="en">')[1].split('<a class')[0].rstrip()
    except IndexError:
        temp_dict['tweet']      = ""
    
    temp_dict['day']        = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[0]
    temp_dict['month']      = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[1]
    temp_dict['year']       = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split('-')[1].lstrip().split(' ')[2]
    temp_dict['time']       = str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[i]).split('title="')[1].split('"><span ')[0].split(' - ')[0]
    
    datetime_object = datetime.strptime(temp_dict['month'] + ' ' + temp_dict['day'] + ' ' + temp_dict['year'] + ' ' + temp_dict['time'].replace(" ", ""), '%b %d %Y %I:%M%p')
    
    temp_dict['datetime']   = datetime_object
    temp_dict['if_tsunami'] = 'tsunami' in temp_dict['tweet'] or 'Tsunami' in temp_dict['tweet'] or '#tsunami' in temp_dict['tweet'] or '#Tsunami' in temp_dict['tweet']
    
    master_list.append(temp_dict)
    print(i)
    

0
1
2
3
4
5
6
7
8
9


In [88]:
master_list[3]

{'handle': 'thejakartaglobe',
 'tweet': 'Indonesian Student Wins Worldwide Comic Contest With Anti-Bullying Hero',
 'day': '14',
 'month': 'Jan',
 'year': '2019',
 'time': '4:39 AM',
 'datetime': datetime.datetime(2019, 1, 14, 4, 39),
 'if_tsunami': False}

In [11]:
pd.DataFrame(master_list).to_csv('bnpb_Indonesia_v2_tweetscrape.csv', sep=",", index=False)

In [ ]:
str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[2])

# #tsunami

In [56]:
hashtag_tweets = bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'})

In [ ]:
list(bs_data.find_all('p', {'class': 'TweetTextSize js-tweet-text tweet-text'}))[0]

In [78]:
if 't-text" data-aria-label-part="0" lang="en">Sadly. Putting the <a class="twitter-hashtag pretty-link js-nav"' in str(hashtag_tweets[0]):
    hashtag_1 = '#' + str(hashtag_tweets[0]).split('</s><b>')[1].split('</b></a>')[0]
hashtag_1
    

'#Indonesian'

In [82]:
str(hashtag_tweets[0])

'<p class="TweetTextSize js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Sadly. Putting the <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/Indonesian?src=hash"><s>#</s><b>Indonesian</b></a> <img alt="🇮🇩" aria-label="Emoji: Flag of Indonesia" class="Emoji Emoji--forText" draggable="false" src="https://abs.twimg.com/emoji/v2/72x72/1f1ee-1f1e9.png" title="Flag of Indonesia"/> <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/tsunami?src=hash"><s>#</s><b><strong>tsunami</strong></b></a>(s) in context: Natural disasters caused $160 billion in damage &amp; 1000s of fatalities in 2018. <a class="twitter-hashtag pretty-link js-nav" data-query-source="hashtag_click" dir="ltr" href="/hashtag/Indonesia?src=hash"><s>#</s><b>Indonesia</b></a> accounted for 3000+ deaths. <a class="twitter-timeline-link" data-expanded-url="https://qz.com/1517738/natural-disasters-caused-160-bi

In [86]:
str(hashtag_tweets[0].get_text())

'Sadly. Putting the #Indonesian  #tsunami(s) in context: Natural disasters caused $160 billion in damage & 1000s of fatalities in 2018. #Indonesia accounted for 3000+ deaths. https://qz.com/1517738/natural-disasters-caused-160-billion-in-damage-in-2018/\xa0… @adamrasmi reports.pic.twitter.com/LCWjlugQV7'

In [75]:
str(hashtag_tweets[0]).split('</s><b>')[1].split('</b></a>')[0]

'Indonesian'

---

# Regular Twitter

In [60]:
list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[0]

<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Gov't Cracks Down on E-Commerce to Raise Tax Collection <a class="twitter-timeline-link" data-expanded-url="http://jglo.be/khg9" dir="ltr" href="https://t.co/CZqXNAzWYm" rel="nofollow noopener" target="_blank" title="http://jglo.be/khg9"><span class="tco-ellipsis"></span><span class="invisible">http://</span><span class="js-display-url">jglo.be/khg9</span><span class="invisible"></span><span class="tco-ellipsis"><span class="invisible"> </span></span></a><a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/aIw6VqztVN">pic.twitter.com/aIw6VqztVN</a></p>

In [149]:
range(loop_range)

range(0, 579)

In [153]:
str(list(bs_data.find_all('a', {'class': 'tweet-timestamp js-permalink js-nav js-tooltip'}))[579]).split('href="/')[1].split('/status')[0]

'thejakartaglobe'

In [155]:
i

5

In [ ]:
str(list(bs_data.find_all('p', {'class': 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text'}))[5]).split('lang="en">')[1].split('<a class')[0].rstrip()